5- La Agencia Nacional de Estadísticas de Buenos Aires recolecta información de nacimientos cuando los padres registran a sus hijos en el registro civil a partir de una encuesta. Esa información se encuentra disponible para su análisis en un csv con el siguiente formato (dia_nacimiento, mes_nacimiento, anio_nacimiento, peso_al_nacer, longitud_al_nacer, id_hospital, tipo_parto), donde el tipo de parto 1 es natural y 2 es cesárea. 
Por otro lado la agencia cuenta con información histórica de los hospitales en otro csv con siguiente formato (id_hospital, dirección, promedio_nacimientos_mensual). Se pide usar Pandas para: 
    A. Calcular la cantidad de nacimientos para cada uno de los hospitales para el mes de Octubre de 2017 e indicar aquellos hospitales que superan el promedio de nacimientos mensuales. 
    B. Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el % de cesáreas con respecto a ese mes del año 2016.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

In [2]:
tam=1000
"""def crearLista(x):
    listaTipos=['transferencia','deposito','extraccion']
    listaReturn=[]
    for i in range(x):
        listaReturn.append(listaTipos[random.randint(0,2)])
    return listaReturn """

def random_date(start, end, tiempo):
    """Generate a random datetime between `start` and `end`"""
    x = start + datetime.timedelta(seconds=random.randint(0, int((end - start).total_seconds())),
    ) 
    dicc = {'dia': x.day,'mes': x.month,'anio':x.year}
    return dicc[tiempo]

m=[]
for x in range (tam): 
    m.append(random_date(pd.to_datetime('1/1/2016'),pd.to_datetime('1/1/2018'),'mes'))
a=[]
for x in range (tam): 
    a.append(random_date(pd.to_datetime('1/1/2016'),pd.to_datetime('1/1/2018'),'anio'))
d=[]
for x in range (tam): 
    d.append(random_date(pd.to_datetime('1/1/2016'),pd.to_datetime('1/1/2018'),'mes'))

hospital = {'dia_nacimiento':d, 
          'mes_nacimiento':m, 
          'anio_nacimiento':a, 
          'peso_al_nacer':np.random.uniform(0.1,8,size=tam), 
          'longitud_al_nacer':np.random.randint(20,50,size=tam), 
          'id_hospital':np.random.randint(100,150,size=tam),
          'tipo_parto':np.random.randint(0,2,size=tam)}
hos = pd.DataFrame(hospital)
hos.head()

,dia_nacimiento,mes_nacimiento,anio_nacimiento,peso_al_nacer,longitud_al_nacer,id_hospital,tipo_parto
0,1,10,2017,0.870977,42,107,0
1,10,2,2017,7.778462,32,127,1
2,9,12,2016,0.571601,40,131,1
3,1,5,2017,7.808369,30,123,0
4,9,12,2017,3.558205,39,147,0


In [3]:
tam=1000
"""def crearLista(x):
    listaTipos=['transferencia','deposito','extraccion']
    listaReturn=[]
    for i in range(x):
        listaReturn.append(listaTipos[random.randint(0,2)])
    return listaReturn """


ebooks = {'direccion':np.random.randint(20,50,size=tam), 
          'id_hospital':random.sample(range(100,100+tam),tam),
          'promedio_nacimientos_mensual':np.random.uniform(0,4,size=tam)}
eb = pd.DataFrame(ebooks)
eb.head()

,direccion,id_hospital,promedio_nacimientos_mensual
0,37,479,2.408913
1,36,645,0.143461
2,43,1090,3.738418
3,36,1022,1.257626
4,47,109,0.980751


A. Calcular la cantidad de nacimientos para cada uno de los hospitales para el mes de Octubre de 2017 e indicar aquellos hospitales que superan el promedio de nacimientos mensuales. 

In [4]:
nwdf=hos.merge(eb,on='id_hospital').groupby(['mes_nacimiento','anio_nacimiento']).get_group((10,2017)).groupby(
    'id_hospital').agg({'promedio_nacimientos_mensual':['count','min']})['promedio_nacimientos_mensual']
nwdf.reset_index().head()

,id_hospital,count,min
0,100,4,0.058281
1,101,3,0.195133
2,102,1,2.626788
3,103,1,0.794665
4,106,1,2.182471


In [5]:
nwdf['supero_promedio']=nwdf.apply(lambda x: x['count']>x['min'],axis=1)

In [6]:
nwdf.drop(nwdf[nwdf['supero_promedio']==False].index,inplace=True)
nwdf.index

Int64Index([100, 101, 103, 109, 115, 126, 134, 148], dtype='int64', name='id_hospital')

B. Comparando el mes de Octubre de 2017 indicar programáticamente si se incremento el % de cesáreas con respecto a ese mes del año 2016.

In [7]:
def calcPorc(mes,anio):
    cdf = hos.groupby(['mes_nacimiento','anio_nacimiento']).get_group((mes,anio)).agg({'tipo_parto':['sum','count']}) 
    return cdf.loc['sum','tipo_parto']/cdf.loc['count','tipo_parto']

True if calcPorc(10,2017)>calcPorc(10,2016) else False

False

0.46153846153846156